In [2]:
# Install necessary libraries
!pip install google-api-python-client


  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached pyasn1-0.6.0-py2.py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -----

In [4]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [5]:
# import libraries
from googleapiclient.discovery import build
import pandas as pd


In [6]:
# setting API key and channel ID
api_key = ''  #ur google api key
channel_id = 'UCX6OQ3DkcsbYNE6H8uQQuVA'              #take any channel id

# Build the YouTube service
youtube = build('youtube','v3',developerKey=api_key)


In [7]:
# function to get channel statistics
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id)
    response = request.execute()
    data = dict(Channel_name=response['items'][0]['snippet']['title'],
                Subscribers=response['items'][0]['statistics']['subscriberCount'],
                Views=response['items'][0]['statistics']['viewCount'],
                Total_videos=response['items'][0]['statistics']['videoCount'],
                Playlist_id=response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    return data

stats = get_channel_stats(youtube, channel_id)
channel_Data = pd.DataFrame([stats])
playlist_id = channel_Data.loc[channel_Data['Channel_name'] == 'MrBeast', 'Playlist_id'].iloc[0]


In [11]:
stats = get_channel_stats(youtube, channel_id)
stats

{'Channel_name': 'MrBeast',
 'Subscribers': '258000000',
 'Views': '48922946378',
 'Total_videos': '794',
 'Playlist_id': 'UUX6OQ3DkcsbYNE6H8uQQuVA'}

In [12]:
channel_Data = pd.DataFrame([stats])
channel_Data

,Channel_name,Subscribers,Views,Total_videos,Playlist_id
0,MrBeast,258000000,48922946378,794,UUX6OQ3DkcsbYNE6H8uQQuVA


In [13]:

playlist_id = channel_Data.loc[channel_Data['Channel_name'] == 'MrBeast', 'Playlist_id'].iloc[0]
playlist_id

'UUX6OQ3DkcsbYNE6H8uQQuVA'

In [8]:
# function to get video IDs
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50)
    response = request.execute()
    video_ids = [item['contentDetails']['videoId'] for item in response['items']]
    
    next_page_token = response.get('nextPageToken')
    while next_page_token:
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token)
        response = request.execute()
        video_ids.extend(item['contentDetails']['videoId'] for item in response['items'])
        next_page_token = response.get('nextPageToken')
        
    return video_ids

video_ids = get_video_ids(youtube, playlist_id)


In [9]:
# function to get video details
def get_video_details(youtube, video_ids):
    all_video_stats = []
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()
        for video in response.get('items', []):
            video_stats = {
                'Title': video['snippet']['title'],
                'Published_date': video['snippet']['publishedAt'],
                'Likes': video['statistics'].get('likeCount', '0'),
                'Views': video['statistics']['viewCount']
            }
            all_video_stats.append(video_stats)
    return all_video_stats



In [14]:
video_details = get_video_details(youtube, video_ids)


In [16]:
videos_data = pd.DataFrame(video_details)
videos_data

,Title,Published_date,Likes,Views
0,Bottle Head Smashing World Record Attempt!,2024-05-14T16:00:00Z,2839280,64996358
1,"Protect The Yacht, Keep It!",2024-05-11T16:00:00Z,3023544,73717881
2,Ages 1-100 Try My Chocolate,2024-05-09T16:00:00Z,3184309,53542137
3,"Spot The Hidden People For $10,000",2024-05-08T16:00:01Z,2598876,63306436
4,"Would You Split Or Steal $250,000?",2024-05-03T16:00:00Z,4018532,47468444
...,...,...,...,...
789,Most Epic minecraft skin EVER (Psy),2013-01-13T01:59:21Z,37479,968664
790,More birds IN MINECRAFT!!,2013-01-12T23:35:45Z,42809,1108911
791,Boxy item mod Minecraft. EPIC,2013-01-12T22:34:11Z,50476,1306413
792,Harry Potter Mod In Minecraft! EPIC MUST SEE M...,2012-03-09T23:29:03Z,0,4763954


In [17]:
videos_data['Published_date'] = videos_data['Published_date'].str.replace('T', ' ').str.split(' ').str[0]


In [18]:
videos_data

,Title,Published_date,Likes,Views
0,Bottle Head Smashing World Record Attempt!,2024-05-14,2839280,64996358
1,"Protect The Yacht, Keep It!",2024-05-11,3023544,73717881
2,Ages 1-100 Try My Chocolate,2024-05-09,3184309,53542137
3,"Spot The Hidden People For $10,000",2024-05-08,2598876,63306436
4,"Would You Split Or Steal $250,000?",2024-05-03,4018532,47468444
...,...,...,...,...
789,Most Epic minecraft skin EVER (Psy),2013-01-13,37479,968664
790,More birds IN MINECRAFT!!,2013-01-12,42809,1108911
791,Boxy item mod Minecraft. EPIC,2013-01-12,50476,1306413
792,Harry Potter Mod In Minecraft! EPIC MUST SEE M...,2012-03-09,0,4763954


In [ ]:
# we can generate the CSV file
file_path = 'youtube_data.csv'
videos_data.to_csv(file_path, index=False)
